In [1]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix, roc_curve
)
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(42)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

Using device: cuda


In [2]:
# Image & Data paths (Update these to your actual paths)
TRAIN_DIR = "/kaggle/input/datasetcavitydetection/Dental_Cavity_Dataset/train"
TEST_DIR = "/kaggle/input/datasetcavitydetection/Dental_Cavity_Dataset/test"

IMG_SIZE = 224
BATCH_SIZE = 32

# Training Phases
NUM_EPOCHS_FROZEN = 5
NUM_EPOCHS_FINETUNE = 30
LR_FROZEN = 1e-3
LR_FINETUNE = 1e-4
PATIENCE = 6

In [3]:
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def get_dataloaders(train_dir, test_dir):
    full_train = datasets.ImageFolder(train_dir, transform=train_transform)
    targets = full_train.targets

    train_idx, val_idx = train_test_split(
        np.arange(len(targets)),
        test_size=0.15,
        stratify=targets,
        random_state=42
    )

    train_ds = Subset(full_train, train_idx)
    val_ds = Subset(
        datasets.ImageFolder(train_dir, transform=val_test_transform),
        val_idx
    )

    test_ds = datasets.ImageFolder(test_dir, transform=val_test_transform)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
    test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

    # Calculate class weights for imbalance
    train_targets = np.array(targets)[train_idx]
    class_counts = np.bincount(train_targets)
    pos_weight = torch.tensor(class_counts[0] / class_counts[1], dtype=torch.float).to(DEVICE)

    return train_loader, val_loader, test_loader, pos_weight

In [4]:
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for x, y in loader:
        x, y = x.to(DEVICE), y.float().to(DEVICE)
        optimizer.zero_grad()
        logits = model(x).squeeze()
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    y_true, y_prob, y_pred = [], [], []
    for x, y in loader:
        x, y = x.to(DEVICE), y.float().to(DEVICE)
        logits = model(x).squeeze()
        probs = torch.sigmoid(logits)
        y_true.extend(y.cpu().numpy())
        y_prob.extend(probs.cpu().numpy())
        y_pred.extend((probs > 0.5).cpu().numpy())

    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred),
        "roc_auc": roc_auc_score(y_true, y_prob),
        "y_true": y_true,
        "y_prob": y_prob
    }

class EarlyStopping:
    def __init__(self, patience=5):
        self.patience = patience
        self.best_loss = np.inf
        self.counter = 0
        self.stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.stop = True

In [5]:
def train_model(model, train_loader, val_loader, pos_weight):
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    early_stopping = EarlyStopping(patience=PATIENCE)

    # Target the head for ConvNeXt, EfficientNet, or ResNet
    head = model.classifier if hasattr(model, 'classifier') else model.fc

    # PHASE 1: Frozen Backbone
    for param in model.parameters(): param.requires_grad = False
    for param in head.parameters(): param.requires_grad = True

    optimizer = optim.Adam(head.parameters(), lr=LR_FROZEN)

    print(f"--- Phase 1: Training {type(model).__name__} Head ---")
    for epoch in range(NUM_EPOCHS_FROZEN):
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
        val_metrics = evaluate(model, val_loader)
        print(f"Epoch {epoch+1}: Loss {train_loss:.4f}, Val AUC {val_metrics['roc_auc']:.4f}")

    # PHASE 2: Fine-tuning
    print(f"\n--- Phase 2: Fine-tuning {type(model).__name__} ---")
    for param in model.parameters(): param.requires_grad = True
    optimizer = optim.AdamW(model.parameters(), lr=LR_FINETUNE)

    for epoch in range(NUM_EPOCHS_FINETUNE):
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
        val_metrics = evaluate(model, val_loader)
        early_stopping(1 - val_metrics['roc_auc']) 
        print(f"FT Epoch {epoch+1}: Loss {train_loss:.4f}, Val AUC {val_metrics['roc_auc']:.4f}")
        if early_stopping.stop:
            print("Early stopping triggered.")
            break

In [6]:
def get_convnext():
    model = models.convnext_tiny(weights=models.ConvNeXt_Tiny_Weights.IMAGENET1K_V1)
    # ConvNeXt classifier index [2] is the linear layer
    in_features = model.classifier[2].in_features
    model.classifier[2] = nn.Linear(in_features, 1)
    return model.to(DEVICE)

In [7]:
# 1. Get Data
train_loader, val_loader, test_loader, pos_weight = get_dataloaders(TRAIN_DIR, TEST_DIR)

# 2. Initialize ConvNeXt-Tiny
conv_model = get_convnext()

# 3. Train
print("🚀 Starting ConvNeXt-Tiny Training Pipeline...")
train_model(conv_model, train_loader, val_loader, pos_weight)

# 4. Final Evaluation
conv_results = evaluate(conv_model, test_loader)

print("\n--- ConvNeXt-Tiny Final Test Results ---")
print(f"Accuracy:  {conv_results['accuracy']:.4f}")
print(f"ROC-AUC:   {conv_results['roc_auc']:.4f}")
print(f"F1-Score:  {conv_results['f1']:.4f}")

Downloading: "https://download.pytorch.org/models/convnext_tiny-983f1562.pth" to /root/.cache/torch/hub/checkpoints/convnext_tiny-983f1562.pth


In [ ]:
import zipfile
import pandas as pd
from IPython.display import FileLink

def generate_results_package(model, test_loader, results_dict, zip_name="ConvNeXt_Dental_Results.zip"):
    # --- 1. Prepare Data for Charts ---
    y_true = results_dict['y_true']
    y_prob = results_dict['y_prob']
    y_pred = [1 if p > 0.5 else 0 for p in y_prob]
    
    sns.set_theme(style="whitegrid")
    generated_files = []

    # --- 2. Confusion Matrix ---
    plt.figure(figsize=(8, 6))
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Healthy', 'Cavity'], yticklabels=['Healthy', 'Cavity'])
    plt.title('Confusion Matrix: ConvNeXt-Tiny')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.savefig('confusion_matrix.png', dpi=300)
    generated_files.append('confusion_matrix.png')
    plt.show()

    # --- 3. ROC Curve ---
    plt.figure(figsize=(8, 6))
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    auc_score = roc_auc_score(y_true, y_prob)
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {auc_score:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.savefig('roc_curve.png', dpi=300)
    generated_files.append('roc_curve.png')
    plt.show()

    # --- 4. Export Metrics to CSV ---
    metrics_data = {
        "Metric": ["Accuracy", "Precision", "Recall", "F1-Score", "ROC-AUC"],
        "Value": [
            results_dict['accuracy'], 
            results_dict['precision'], 
            results_dict['recall'], 
            results_dict['f1'], 
            results_dict['roc_auc']
        ]
    }
    df = pd.DataFrame(metrics_data)
    df.to_csv('final_metrics.csv', index=False)
    generated_files.append('final_metrics.csv')

    # --- 5. Save Model Weights ---
    torch.save(model.state_dict(), 'convnext_dental_model.pth')
    generated_files.append('convnext_dental_model.pth')

    # --- 6. Zip Everything ---
    with zipfile.ZipFile(zip_name, 'w') as zipf:
        for file in generated_files:
            if os.path.exists(file):
                zipf.write(file)
                print(f"✅ Added {file} to zip.")
    
    print(f"\n🚀 Package Ready: {zip_name}")
    return FileLink(zip_name)

# Run the final packager
generate_results_package(conv_model, test_loader, conv_results)